In [3]:
import pandas as pd
from collections import Counter
import numpy as np
pd.options.display.max_columns=None
pd.options.display.max_colwidth=None

In [4]:
file_path = "../data/en.openfoodfacts.org.products.csv.gz"

In [24]:
chunk_size = 10000

filtered_chunks = []

for chunk in pd.read_csv(
    file_path,
    chunksize=chunk_size,
    compression="gzip",
    sep="\t",
    engine="python",
    quoting=3,
):
    filtered_chunks_en = chunk[
        (chunk["countries_tags"] == "en:france") & (chunk["ingredients_tags"].notnull())
    ]
    filtered_chunks.append(filtered_chunks_en)

if filtered_chunks:
    data_en = pd.concat(filtered_chunks, axis=0)

In [ ]:
# Find the different state of each rows
states_unique_values = set()
data_en["states_tags"].str.split(",").apply(states_unique_values.update)

states_df = pd.DataFrame(data=states_unique_values, columns=["states"])
display(states_df)

,states
0,en:packaging-completed
1,en:ingredients-photo-to-be-selected
2,en:quantity-to-be-completed
3,en:origins-to-be-completed
4,en:checked
5,en:characteristics-to-be-completed
6,en:to-be-completed
7,en:nutrition-photo-selected
8,en:photos-to-be-validated
9,en:brands-to-be-completed


In [26]:
# Target sta
target_states = [
    "en:completed",
    "en:origins-completed",
    "en:ingredients-completed",
    "en:characteristics-completed",
]

# Construire une expression régulière pour les états
pattern = "|".join(target_states)

# Compter les lignes contenant au moins un des états cibles
count = data_en["states_tags"].str.contains(pattern).sum()

print("Nombre de lignes avec les états spécifiés:", count)

data_en_filtered = data_en[data_en["states_tags"].str.contains(pattern)]

Nombre de lignes avec les états spécifiés: 258361


In [27]:
len(data_en) - 258361

4085

In [28]:
describe_sample = data_en_filtered.describe(include="all").drop("top").T

In [29]:
describe_sample.T

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_en,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutrition_data,additives_n,additives,additives_tags,additives_en,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,food_groups,food_groups_tags,food_groups_en,states,states_tags,states_en,brand_owner,ecoscore_score,ecoscore_grade,nutrient_levels_tags,product_quantity,owner,data_quality_errors_tags,unique_scans_n,popularity_tags,completeness,last_image_t,last_image_datetime,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,unsaturated-fat_100g,monounsaturated-fat_100g,omega-9-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,omega-6-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,added-sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,erythritol_100g,fiber_100g,soluble-fiber_100g,insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,added-salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,sulphate_100g,nitrate_100g,acidity_100g
count,258361,258361,258360,258361.0,258361,258361.0,258361,252262,258301.0,258301,255537,14319,72675,191654,130790,130790,130790,12117,243033,243026,222035,222035,222035,50696,50660,50659,61964,61940,160704,160704,160704,61828,61817,48192,0.0,51561,77107,112989,258361,258361,258361,258360,258361,258359,100665,0.0,59935,77609,76653,65049,63101.0,13357,258361.0,2,131893,131893,191513.0,258345,224009.0,258360,258361,200085,200085,200085,258361,258361,258361,3877,159886.0,258353,193750,190191.0,50670,8444,188487.0,185127,258361.0,252152.0,252152,222035,222035,239625,239625,224313,224313,183187,183187,85813.0,213971.0,231800.0,28.0,231549.0,229837.0

In [30]:
threshold = 0.85
fill_threshold_percent = describe_sample[describe_sample["count"] > len(data_en_filtered) * threshold].T.columns
fill_threshold_percent

Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'last_modified_by',
       'last_updated_t', 'last_updated_datetime', 'product_name', 'brands',
       'brands_tags', 'categories', 'categories_tags', 'categories_en',
       'countries', 'countries_tags', 'countries_en', 'ingredients_text',
       'ingredients_tags', 'ingredients_analysis_tags', 'additives_n',
       'nutriscore_grade', 'nova_group', 'pnns_groups_1', 'pnns_groups_2',
       'states', 'states_tags', 'states_en', 'ecoscore_grade', 'completeness',
       'last_image_t', 'last_image_datetime', 'main_category',
       'main_category_en', 'image_url', 'image_small_url',
       'image_ingredients_url', 'image_ingredients_small_url', 'energy_100g',
       'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g',
       'proteins_100g', 'salt_100g', 'sodium_100g',
       'fruits-vegetables-nuts-estimate-from-ingredients_100g'],
      dtype='object'

In [31]:
columns_to_keep = [
    "code",
    "url",
    "product_name",
    "quantity",
    "packaging",
    "packaging_en",
    "categories",
    "categories_tags",
    "labels_en",
    "ingredients_text",
    "ingredients_tags",
    "ingredients_analysis_tags",
    "allergens",
    "additives_tags",
    "traces_tags",
    "nutriscore_score",
    "ecoscore_score",
    "image_url",
    "image_small_url",
    "energy-kcal_100g",
    "energy_100g",
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "proteins_100g",
    "salt_100g",
    "sodium_100g",
    "fruits-vegetables-nuts-estimate-from-ingredients_100g",
]
data_en_filtered = data_en_filtered[columns_to_keep]

In [32]:
print(len(data_en_filtered))
# Remove deplicates lines from the dataset
data_en_filtered = data_en_filtered.drop_duplicates()
print(len(data_en_filtered))

258361
257718


In [33]:
print(len(data_en_filtered))
# Remove the duplicates code in the dataset
data_en_filtered = data_en_filtered.drop_duplicates(subset="code")
print(len(data_en_filtered))

257718
257585


In [35]:
# Save the data_en_filtered
data_en_filtered.to_csv("../data/filtered_dataset_openfoodfacts_en.csv", index=False)

### Data Cleaning

In [5]:
data_en_filtered = pd.read_csv("../data/filtered_dataset_openfoodfacts_en.csv", dtype={'code': str})

In [6]:
def catetegorial_and_numerical_features(df):
    categorical_features = df.select_dtypes(include=['object']).columns
    numerical_features = df.select_dtypes(include=['number']).columns
    return categorical_features, numerical_features

In [7]:
categorical_feature, numerical_feature = catetegorial_and_numerical_features(data_en_filtered)

In [8]:
to_split = [
    "packaging_en",
    "categories_tags",
    "ingredients_tags",
    "ingredients_analysis_tags",
    "allergens",
    "additives_tags",
    "traces_tags",
]

In [9]:
for feature in to_split:
    data_en_filtered[feature] = data_en_filtered[feature].fillna("").str.replace("en:", "").str.split(",")

In [10]:
def select_frequent_tags(data, column_name, min_count=100, min_for_various_tags=100):
    all_tags = [item for items in data[column_name].dropna() for item in items]
    tag_counts = Counter(all_tags)
    frequent_tags = [tag for tag, count in tag_counts.items() if count >= min_count]
    if len(frequent_tags) > min_for_various_tags:
        frequent_tags = [tag for tag, _ in tag_counts.most_common(min_for_various_tags)]
    return frequent_tags

In [11]:
# Applique cette fonction aux colonnes de to_split
frequent_tags = {col: select_frequent_tags(data_en_filtered, col) for col in to_split}

In [24]:
np.sum([len(i) for i in frequent_tags.values()])

np.int64(451)

In [25]:
# Transformation en colonnes booléennes seulement pour les tags fréquents
new_columns = {}

for col, tags in frequent_tags.items():
    for tag in tags:
        new_columns[f"{col}_{tag}"] = data_en_filtered[col].apply(
            lambda x: tag in x if isinstance(x, list) else False
        )

# Convert the dictionary to a DataFrame
new_columns_df = pd.DataFrame(new_columns)

In [10]:
print(data_en_filtered.shape)

print(new_columns_df.shape)

(257585, 29)
(257585, 451)


In [11]:
# data_en_filtered = pd.concat([data_en_filtered, new_columns_df], axis=1)

In [12]:
new_columns_df

,packaging_en_,packaging_en_Plastic,packaging_en_Cardboard,packaging_en_Bag,packaging_en_Fresh,packaging_en_Glass,packaging_en_Tray,packaging_en_Box,packaging_en_Bottle,packaging_en_Metal,packaging_en_Frozen,packaging_en_Pot,packaging_en_Jar,packaging_en_Canned,packaging_en_fr:film-en-plastique,packaging_en_Protective gas,packaging_en_fr:sachet-plastique,packaging_en_fr:etui-en-carton,packaging_en_Film,packaging_en_Paper,packaging_en_Brick,packaging_en_Sleeve,packaging_en_Vacuum-packed,packaging_en_Lid,packaging_en_Recyclable Metals,packaging_en_Aluminium,packaging_en_fr:film-plastique,packaging_en_Seal,packaging_en_fr:sachet-en-plastique,packaging_en_Bottle cap,packaging_en_Wine cork,packaging_en_fr:barquette-plastique,packaging_en_fr:bocal-en-verre,packaging_en_Vial,packaging_en_Can,packaging_en_fr:pot-en-verre,packaging_en_fr:point-vert,packaging_en_fr:0,packaging_en_fr:bouteille-en-verre,packaging_en_Pack,packaging_en_Dry,packaging_en_fr:pensez-au-tri,packaging_en_fr:boite-en-carton,packaging_en_fr:Conteneur,packaging_en_0,packaging_en_fr:boite-carton,packaging_en_fr:pot-en-plastique,packaging_en_Pouch flask,packaging_en_fr:couvercle-en-metal,packaging_en_fr:etui-carton-a-recycler,packaging_en_Drink can,packaging_en_fr:etui-carton,packaging_en_fr:bocal-verre,packaging_en_fr:pot-plastique,packaging_en_fr:bouteille-plastique,packaging_en_fr:bouteille-en-plastique,packaging_en_fr:bouteille-verre,packaging_en_Capsule,packaging_en_Tetra Pak,packaging_en_Container,packaging_en_Envelope,packaging_en_fr:opercule-en-plastique,packaging_en_fr:couvercle-metal,packaging_en_fr:opercule-en-metal,packaging_en_fr:capsule-en-metal,packaging_en_Wood,packaging_en_fr:barquette-en-plastique,packaging_en_fr:triman,packaging_en_fr:sachet-plastique-a-jeter,packaging_en_fr:brique-en-carton,packaging_en_fr:bouchon-en-plastique,packaging_en_fr:film-plastique-a-jeter,packaging_en_Green-dot,packaging_en_Individual bag,packaging_en_Recycle,packaging_en_Bottle or vial,packaging_en_Refrigerated,packaging_en_Wrapper,packaging_en_fr:boite-plastique,packaging_en_fr:boite-en-metal,packaging_en_Sheet,packaging_en_Blister,packaging_en_fr:barquette-et-film-plastique-a-jeter,packaging_en_Food can,packaging_en_Unfrozen,packaging_en_Discard,packaging_en_fr:pot-verre,packaging_en_fr:bouchon-plastique,packaging_en_Pet-polyethylene-terephthalate,packaging_en_Steel,packaging_en_Net,packaging_en_fr:boite-metal,packaging_en_fr:couvercle-plastique,packaging_en_Tube,packaging_en_Plastique,packaging_en_Card-box,packaging_en_fr:sachet-papier,packaging_en_fr:couvercle-ouverture-facile,packaging_en_fr:flacon-verre,packaging_en_Jug,categories_tags_plant-based-foods-and-beverages,categories_tags_plant-based-foods,categories_tags_snacks,categories_tags_,categories_tags_sweet-snacks,categories_tags_meats-and-their-products,categories_tags_beverages,categories_tags_meals,categories_tags_dairies,categories_tags_fruits-and-vegetables-based-foods,categories_tags_cereals-and-potatoes,categories_tags_meats,categories_tags_desserts,categories_tags_fermented-foods,categories_tags_biscuits-and-cakes,categories_tags_fermented-milk-products,categories_tags_frozen-foods,categories_tags_spreads,categories_tags_prepared-meats,categories_tags_cereals-and-their-products,categories_tags_condiments,categories_tags_breakfasts,categories_tags_fruits-based-foods,categories_tags_cheeses,categories_tags_plant-based-beverages,categories_tags_groceries,categories_tags_seafood,categories_tags_confectioneries,categories_tags_cocoa-and-its-products,categories_tags_vegetables-based-foods,categories_tags_dairy-desserts,categories_tags_canned-foods,categories_tags_biscuits,categories_tags_sauces,categories_tags_sweet-spreads,categories_tags_fishes,categories_tags_fishes-and-their-products,categories_tags_plant-based-spreads,categories_tags_salty-snacks,categories_tags_fermented-dairy-desserts,categories_tags_fruit-based-beverages,categories_tags_chocolates,categories_tags_meals-with-meat,categories_tags_cakes,

In [ ]:
describe_sample = data_en_filtered.describe(include="all").drop("top").T